In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from display_config import enable_clean_display
from db_connector import fetch_data

In [ ]:
enable_clean_display()  # configures horizontal scrolling for wide DataFrames -- wasn't happening through built-in options

### Note:  
Sales tables, geographical data, and UOM names are in separate schemas, and cross-schema joins are not allowed in PostgreSQL.</br>
So read queries on each schema into their own DataFrames, then merge.

In [ ]:
sales_df = fetch_data('''
    SELECT
        il.invoice_line_id,
        il.invoice_id,
        il.stock_item_id,
        il.quantity,
        il.package_type_id,
        il.unit_price,
        il.extended_price,
        o.order_date,
        o.expected_delivery_date,
        i.confirmed_delivery_time,
        c.delivery_city_id
    FROM sales.invoice_lines il
        INNER JOIN sales.invoices i
            ON il.invoice_id = i.invoice_id
        INNER JOIN sales.orders o
            ON i.order_id = o.order_id
        INNER JOIN sales.customers c
            ON o.customer_id = c.customer_id;
''')

print(sales_df.info())

In [ ]:
geo_df = fetch_data('''
    SELECT
        c.city_id,
        c.city_name,
        sp.state_province_code,
        sp.state_province_name,
        sp.sales_territory
    FROM application.cities c
        JOIN application.state_provinces sp
            ON c.state_province_id = sp.state_province_id;
''').rename(columns={
    'state_province_code': 'state_code',
    'state_province_name': 'state_name'
})

print(geo_df.info())

In [ ]:
uom_df = fetch_data('''
    SELECT
        package_type_id,
        package_type_name
    FROM warehouse.package_types;
''')

print(uom_df.info())

In [ ]:
# The join columns between all 3 tables are the same dtype (int64), so proceed with the merge
df = (pd.merge(sales_df, geo_df, 
    left_on='delivery_city_id',
    right_on='city_id',
    how='inner')
    .drop(columns=['city_id'])
)

df = (pd.merge(df, uom_df,
    left_on='package_type_id',
    right_on='package_type_id',
    how='inner')
)

print(df.info())

</br>Convert column dtypes where needed:
</br> * All dates and date-times should be 'datetime64'. Opt to use pd.to_datetime() to avoid errors.
</br> * All ID columns should be Pandas 'Int64'.
</br> * All monetary amounts should be 'float64'.
</br> * Suitable categorical columns should be 'category'.
</br>
</br>Set index to the GUID, 'invoice_line_id'.

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['expected_delivery_date'] = pd.to_datetime(df['expected_delivery_date'])

dtype_dict = {
    'invoice_line_id': 'Int64',
    'invoice_id': 'Int64',
    'stock_item_id': 'Int64',
    'quantity': 'Int64',
    'package_type_id': 'Int64',
    'delivery_city_id': 'Int64',
    'city_name': 'category',
    'state_code': 'category',
    'state_name': 'category',
    'sales_territory': 'category',
    'package_type_name': 'category'
}

df = df.astype(dtype_dict).set_index('invoice_line_id')

df.info()

In [ ]:
# Check for missing values
df.isna().sum()

### Note:  
Since I may analyze aspects of the data that don't involve deliveries,</br>
keep all 284 rows with missing datetimes in the base table and filter out in summary table(s).

In [ ]:
# Check for duplicates index values
print(df.index.duplicated().sum())

In [ ]:
# Check for entire duplicate rows, excluding index
print(df.duplicated().sum())

In [ ]:
# Create date and time metric columns

# Extract date components
df['order_mo'] = df['order_date'].dt.month
df['order_yr'] = df['order_date'].dt.year

# Calculate delivery durations
df['cycle_time_days'] = (
    df['confirmed_delivery_time'] - df['order_date']
    ).dt.days.astype('Int64')

df['delivery_variance_days'] = (
    df['confirmed_delivery_time'] - df['expected_delivery_date']
    ).dt.days.astype('Int64')

In [ ]:
# Create order volume metric columns, measured at the invoice/order level

# Calculate a metric for order magnitude
df['total_order_qty'] = df.groupby('invoice_id')['quantity'].transform('sum')

# Calculate a metric for order complexity -- defined as number of lines in an order
df['line_item_count'] = df.groupby('invoice_id')['quantity'].transform('count')

# Calculate the average quantity per line within each order
df['avg_qty_per_line'] = (df['total_order_qty'] / df['line_item_count']).round(2)

In [ ]:
# Check for outliers in the key metric columns
print(df[[
    'cycle_time_days', 'delivery_variance_days',
    'total_order_qty', 'line_item_count',
    'avg_qty_per_line'
    ]].describe())

### Note:  
Only create an outlier flag if it directly supports answering the initial summary question.  
If it doesn’t help isolate the problem, delete it.

In [ ]:
# Create column to flag extremely late deliveries
# Business Logic: Target is 0 days variance; 3 days late is unacceptable
df['is_late_delivery'] = df['delivery_variance_days'] >= 3.0
print(df[df['is_late_delivery'] == True]['invoice_id'].count())

### Outlier Flagging
* `is_late_delivery`: ≥ 3 days variance - KEPT (2,828 rows)
* All other flags DELETED (not relevant to root cause)

In [ ]:
# See if any flagged outliers are the same records with missing confirmed_delivery_time values

# First create a table only containing extremely late deliveries
df_late = df[df['is_late_delivery'] == True]

print(df_late['confirmed_delivery_time'].isna().sum())

### Note:  
No rows that are flagged as extremely late have missing confirmed_delivery_time values.

## Initial Question:
* There's an extreme delivery variance outlier of 1,114 days -- over 3 years
* What segmentation of this data will give more insight into the extremely late deliveries?

In [ ]:
# Find which category column best explains the extremely late deliveries

# List of categorical columns to test
category_cols = ['stock_item_id', 'sales_territory', 'package_type_name', 'city_name', 'state_name']

# For each column, measure how much it concentrates late deliveries
rates = {}
for column in category_cols:
    if column in df.columns:
        # % of rows in this category that is a late delivery
        df_late_rate = df_late.groupby(column, observed=False).size() / df.groupby(column, observed=False).size()
        # Max rate: highest % of late deliveries in any single value
        df_max_rate = df_late_rate.max()
        rates[column] = df_max_rate

# Show results
pd.Series(rates).sort_values(ascending=False).round(3)

## Initial Finding:
* Grouping by stock_item_id yields the highest rate of extremely late deliveries.

## Follow-Up Question:
* Which stock items have the most extremely late deliveries?

In [ ]:
# Identify the problem items

# First group by SKU and sum the order lines where delivery was extremely late
df_late_sums = df.groupby('stock_item_id', observed=False)['is_late_delivery'].sum()

# Then filter for only items where extremely late deliveries occurred; sort high to low
df_late_items = df_late_sums[df_late_sums > 0].sort_values(ascending=False)

print(df_late_items)

In [ ]:
# Visualize the count of extremely late deliveries among problem items
ax1 = (
    df_late_items
    .sort_values()
    .plot.barh(
        title='Items with Extremely Late Deliveries',
        xlabel='Late Delivery Count',
        ylabel='Stock Item ID',
        color='Tan',
    )
);

# Save chart to file for reporting
ax1.figure.savefig('images/late_items_count.png')

## Follow-Up Question:
* On average, how late are the deliveries on these items?  
(Use median to avoid the skew.)

In [ ]:
# Analyze the severity of late delivery for each problem item

# First, isolate the problem item SKUs for easy filtering
problem_items = df_late_items.index.to_list()

# Create a focused table of invoice lines for only problem items - simplifies drill-down
df_problem_items = df[df['stock_item_id'].isin(problem_items)]

# Calculate the median delivery variance for the problem items
df_med_late_delivery = df_problem_items.groupby('stock_item_id')['delivery_variance_days'].median().sort_values(ascending=False)

print(df_med_late_delivery)

In [ ]:
# Visualize the median delivery variance among problem items
ax2 = (
    df_med_late_delivery
    .plot.bar(
        title='Median Delivery Variance of Problem Items',
        xlabel='Stock Item ID',
        ylabel='Median Delivery Variance (Days)',
        rot=0,
        color='Tan',
    )
);

# Save chart to file for reporting
ax2.figure.savefig('images/median_delivery_variance.png')

## Initial  Summary Analysis

**Discovered Performance Driver:**  
> Nine stock items account for all extremely late deliveries:
</br> - IDs:  `95`, `80`, `204`, `193`, `77`, `98`, `78`, `86`, `184`  

> ID `95` is routinely late by a full week or more  

> ID `184` is only infrequently late by an extreme amount

## Secondary Question:  
* What is special about these nine stock items that causes such long delivery delays?

In [ ]:
# Look for clustering of problem items in other category and metric columns

# Test hypotheses
print('\nLate Deliveries by Package Type')
print(df_problem_items
      .groupby('package_type_name', observed=False)
      .size()
      .sort_values(ascending=False))

print('\nLate Deliveries by Sales Territory')
print(df_problem_items.
      groupby('sales_territory', observed=False)
      .size()
      .sort_values(ascending=False))

print('\nLate Deliveries by Avg Qty per Line')
print(df_problem_items
      .groupby('avg_qty_per_line', observed=False)
      .size()
      .sort_values(ascending=False)
      .head(20))

## Diagnostic Drill-Down

**Root Cause:**
</br> - All 6,041 unacceptably late deliveries involve stock items sold in "Each" units.

**Evidence:**
</br> - `package_type_name == "Each"`: 6,041 unacceptably late deliveries
</br> - All other types (`Bag`, `Packet`, `Pair`): 0 unacceptably late deliveries
</br> - No concentration among territories (Southeast leads but doesn't indicate anomaly)

**Secondary Question Answered:**
</br> - What is special about these nine stock items?
</br> --> They are all sold in "Each" units — so "Each" packaging causes 100% of unacceptably late deliveries.

## Recommended Actions  
  
Since estimated delivery dates are calculated after the order is placed,  
the delivery failures are not a factor of upstream issues like vendor delays or stocking issues.
1. Audit 9 SKUs for delivery forecasting issues
</br> - Owner: Supply Chain
2. Review "Each" packaging: Do they require special handling? Is it an issue of fragility?
</br> - Owner: Logistics
3. Add UOM-based SLA for "Each" items and monitor proactively
</br> - Owner: Operations

---

**Business Impact**:  
* Fixing these 9 items will eliminate all unacceptably late deliveries  